In [1]:
import pandas as pd
import numpy as np
import yaml

In [6]:
# Ruta a tu archivo YAML
ruta = r"C:\Users\scerda\Documents\langgraph-postgres-energy-data\src\agent\schema_context.yaml"

# Leer el archivo como string y convertirlo a dict
with open(ruta, 'r', encoding='utf-8') as f:
    schema = yaml.safe_load(f)

In [7]:
schema

{'schema': 'smart_buildings',
 'description': 'Contains energy consumption data for smart buildings',
 'tables': [{'name': 'building',
   'description': 'Información de edificios; útil para identificar y categorizar los edificios.',
   'columns': [{'name': 'cups',
     'type': 'string',
     'description': 'Código Universal del Punto de Suministro; clave única para cada edificio.'},
    {'name': 'name',
     'type': 'string',
     'description': 'Nombre del edificio; útil para identificación y presentación.'},
    {'name': 'address',
     'type': 'string',
     'description': 'Dirección del edificio; útil para geolocalización y logística.'},
    {'name': 'type',
     'type': 'string',
     'description': 'Tipo de edificio (Administración, Comercial, etc.); útil para segmentación y análisis.'}]},
  {'name': 'energy_consumption_monthly_metrics',
   'description': 'Métricas de consumo eléctrico agregadas por mes civil (yyyy-MM-01).',
   'columns': [{'name': 'cups',
     'type': 'string',


In [3]:
# get current date
from datetime import datetime
from datetime import timedelta

date = datetime.now()

In [14]:
import pandas as pd

df = pd.read_csv("consultas_edificio.csv", sep=";", encoding="utf-8")
df_markdown=df.to_markdown(index=False)

In [19]:
prompt = f"""You are a data engineer tasked with generating a dataset to train a model that maps natural language questions to SQL queries.

Given the database schema below:

{schema}

And the following examples for reference:

{df_markdown}

Generate at least 100 new, diverse pairs of:
- a natural language question (English)
- its corresponding SQL query that correctly answers the question

Return the output as a valid CSV with two columns: `question` and `sql`. Do not include explanations or any formatting outside the CSV. Escape quotes and line breaks appropriately for CSV format.

The SQL queries must be syntactically correct and executable under the given schema. Avoid overly repetitive questions, and include a range of query types such as selections, filters, group by, order by, joins (if the schema allows), and aggregations.
The questions must be on proper Spanish.
"""

In [20]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
  model="gpt-4.1-mini",
  input=prompt
)





In [21]:
print(response.output_text)

question,sql
"¿Cuál es el consumo total mensual del edificio Biblioteca Central en enero de 2024?","SELECT b.""name"" AS building_name, m.""year_month"", m.""total_consumption_kwh"" FROM smart_buildings.""building"" b JOIN smart_buildings.""energy_consumption_monthly_metrics"" m ON b.""cups"" = m.""cups"" WHERE b.""name"" = 'Biblioteca Central' AND m.""year_month"" = DATE '2024-01-01';"
"¿Cuál fue el consumo diario promedio del edificio Torre Ejecutiva en marzo de 2025?","SELECT b.""name"" AS building_name, m.""year_month"", m.""daily_consumption_kwh"" FROM smart_buildings.""building"" b JOIN smart_buildings.""energy_consumption_monthly_metrics"" m ON b.""cups"" = m.""cups"" WHERE b.""name"" = 'Torre Ejecutiva' AND m.""year_month"" = DATE '2025-03-01';"
"¿Qué edificio tuvo el mayor consumo total en junio de 2024?","SELECT b.""name"" AS building_name, m.""year_month"", m.""total_consumption_kwh"" FROM smart_buildings.""building"" b JOIN smart_buildings.""energy_consumption_monthly_metri

In [ ]:
with open("output.csv", "w", encoding="utf-8") as f:
    f.write(response.output_text)

In [26]:
df = pd.read_excel("consultas_edificio.xlsx")
df.head()

,Tipo,pregunta,relevant_values,consulta_sql
0,Consumo acumulado YTD,¿Y cómo se compara ese YTD con el año pasado e...,NaN,"SELECT b.""name"" AS building_name, m.""year_mont..."
1,Consumo Total Mensual,¿Y cómo se compara con el mes anterior?,NaN,"SELECT b.""name"" AS building_name, m.""year_mont..."
2,Consumo Total Mensual,Cual es el edificio con mayor consumo en abril...,NaN,"SELECT b.""name"" AS building_name, m.""year_mont..."
3,Consumo Total Mensual,Cual es el edificio con menor consumo en abril...,NaN,"SELECT b.""name"" AS building_name, m.""year_mont..."
4,Consumo total semanal,¿Cuál es el consumo total de energía de todos ...,NaN,"SELECT w.""week_start"", SUM(w.""total_consumptio..."


In [ ]:
import pandas as pd
from openai import OpenAI
import time
from tqdm import tqdm
# Cargar datos
df = pd.read_excel("consultas_edificio.xlsx")

# Inicializar cliente de OpenAI
client = OpenAI()

# Función para generar reformulaciones
def generate_rephrasings(question: str) -> list:
    prompt = f"""
    Eres un asistente de datos que ayuda a generar reformulaciones alternativas de preguntas de usuario.
Dada la siguiente pregunta:

"{question}"

Genera 3 versiones diferentes:
- Mantén el significado central intacto.
- Usa al menos una versión con mala gramática o errores de ortografía.
- Si hay entidades nombradas (como nombres de edificios), intencionalmente mándalas mal en 1 o 2 casos.
- Haz que cada pregunta suene como si la hubiera hecho una persona diferente.
- Usa diferentes estilos de lenguaje (formal, coloquial, etc.).
- Evita repetir la misma estructura de pregunta.


Return only the rephrased questions as a Python list of strings.
"""
    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt
        )
        text = response.output_text.strip()
        if text.startswith("```") and text.endswith("```"):
            text = "\n".join(text.split("\n")[1:-1])  # Remove first and last lines

        # Evaluar el output como lista
        return eval(text)
    except Exception as e:
        print(f"Error with question: {question}\n{e}")
        return []

# Lista para almacenar resultados
all_rows = []

# Iterar sobre cada fila del DataFrame
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    original_question = row["pregunta"]
    rephrasings = generate_rephrasings(original_question)

    # Agregar la original
    all_rows.append({
        "question": original_question,
        "type": row["Tipo"],
        "sql": row["consulta_sql"],
        "relevant_values": row["relevant_values"],
        "variation": "original"
    })

    # Agregar las reformuladas
    for i, q in enumerate(rephrasings):
        all_rows.append({
            "question": q,
            "type": row["Tipo"],
            "sql": row["consulta_sql"],
            "relevant_values": row["relevant_values"],
            "variation": f"rephrased_{i+1}"
        })



# Crear nuevo DataFrame
df_expanded = pd.DataFrame(all_rows)

# Guardar en CSV
df_expanded.to_csv("rephrased_questions.csv", index=False)


Processing rows: 100%|██████████| 120/120 [05:09<00:00,  2.58s/it]


In [3]:
import pandas as pd
# Cargar tu DataFrame desde el CSV
df = pd.read_csv('training.csv',sep=";")  # Ajusta nombre archivo

df

,pregunta,sql,schema
0,¿Cuál fue el consumo energético total del edif...,SELECT e.total_consumption_kwh FROM smart_buil...,CREATE TABLE smart_buildings.building (\n c...
1,¿Cuántos kWh consumió el edificio ‘Torres Nort...,SELECT e.total_consumption_kwh FROM smart_buil...,CREATE TABLE smart_buildings.building (\n c...
2,¿Cuál fue el consumo promedio diario del edifi...,SELECT e.avg_daily_consumption_kwh FROM smart_...,CREATE TABLE smart_buildings.building (\n c...
3,¿Me puedes indicar la desviación estándar del ...,SELECT e.std_daily_consumption_kwh FROM smart_...,CREATE TABLE smart_buildings.building (\n c...
4,"Cuanto ha sido el consumo de ""Deportivo Unido""...","SELECT e.year_month, e.total_consumption_kwh F...",CREATE TABLE smart_buildings.building (\n c...
5,Cual fue el cosumo promedio del edificio Regio...,SELECT e.avg_daily_consumption_kwh FROM smart_...,CREATE TABLE smart_buildings.building (\n c...
6,Cual fue el promedio de consumo semanal en el ...,SELECT ROUND(AVG(e.total_consumption_kwh)::num...,CREATE TABLE smart_buildings.building (\n c...
7,¿Cuánto fue el consumo energético total del ed...,SELECT e.total_consumption_kwh FROM smart_buil...,CREATE TABLE smart_buildings.building (\n c...
8,"¿Me puedes dar el consumo total, el promedio m...",SELECT SUM(e.total_consumption_kwh) AS total_c...,CREATE TABLE smart_buildings.building (\n c...
9,"¿Cuál fue el consumo total, el consumo del mes...","SELECT e.year_month, e.total_consumption_kwh, ...",CREATE TABLE smart_buildings.building (\n c...


In [16]:
import pandas as pd
from openai import OpenAI
import time

client = OpenAI()

# Cargar tu DataFrame desde el CSV
df = pd.read_csv('training.csv',sep=";")  # Ajusta nombre archivo

def construir_prompt(filas):
    # Construye el prompt de few-shot para el modelo
    ejemplo = "\n".join([
        f"Pregunta: {row['pregunta']}\nSQL: {row['sql']}\n"
        for idx, row in filas.iterrows()
    ])
    prompt = (
        f"Estos son ejemplos de preguntas sobre consumo energético de edificios y su respectivo SQL.\n"
        f"Parafrasea y genera 10 nuevas preguntas y su SQL, cambiando el nombre de las entidades (edificios, fechas) respecto a los ejemplos.\n "
        f"No inventes variables que no existen, usa las mismas variables que puedes ver en las consultas de SQL\n"
        f"Mantén la misma lógica en el SQL pero ajusta los nombres/fechas, y asegúrate que cada pregunta y su SQL correspondan.\n"
        f"Devuelve el resultado en formato CSV: question;sql\n\n"
        f"{ejemplo}\n"
        f"Genera ahora 5 nuevos pares:"
    )
    return prompt

all_new_rows = []
for i in range(0, len(df), 5):
    bloque = df.iloc[i:i+5]
    prompt = construir_prompt(bloque)
    
    try:
        response = client.responses.create(
            model="gpt-4.1",
            input=prompt
        )
        # Extraer el texto de la respuesta
        output = response.output_text
        # Parsear líneas tipo "pregunta;sql"
        for linea in output.split('\n'):
            if ';' in linea:
                q, s = linea.split(';', 1)
                all_new_rows.append({'pregunta': q.strip(), 'sql': s.strip()})
    except Exception as e:
        print(f"Error en bloque {i}: {e}")
       


# Crear DataFrame y unirlo
df_new = pd.DataFrame(all_new_rows)
df_final = pd.concat([df, df_new], ignore_index=True)

# Guardar el resultado
df_final.to_csv('training_generated.csv', index=False)


In [17]:
print(response.output_text)

question;sql
¿Cuáles son los 8 edificios con mayor consumo de energía durante 2024?;SELECT b.name, SUM(e.total_consumption_kwh) AS total_consumption_kwh FROM smart_buildings.energy_consumption_monthly_metrics e JOIN smart_buildings.building b ON e.cups = b.cups WHERE EXTRACT(YEAR FROM e.year_month) = 2024 GROUP BY b.name ORDER BY total_consumption_kwh DESC LIMIT 8
¿Cuál es el top 6 de equipamientos con menor cambio porcentual respecto al mes anterior?;SELECT b.name, e.year_month, e.diff_pct_consumption_prev_month FROM smart_buildings.energy_consumption_monthly_metrics e JOIN smart_buildings.building b ON e.cups = b.cups WHERE e.year_month = date_trunc('month', CURRENT_DATE)::date AND e.diff_pct_consumption_prev_month IS NOT NULL ORDER BY ABS(e.diff_pct_consumption_prev_month) ASC LIMIT 6
¿Cuáles son los 4 equipamientos de Escuelas con mayor consumo en el año vigente?;SELECT b.name, SUM(e.total_consumption_kwh) AS total_consumption_kwh FROM smart_buildings.energy_consumption_monthly_met